In [1]:
'''Trains a simple deep NN on the MNIST dataset.
Gets to 98.40% test accuracy after 20 epochs
(there is *a lot* of margin for parameter tuning).
2 seconds per epoch on a K520 GPU.
'''

from __future__ import print_function

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

Using TensorFlow backend.


[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7925991087406905591
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 22654317364
locality {
  bus_id: 1
}
incarnation: 481532239106186608
physical_device_desc: "device: 0, name: Tesla P40, pci bus id: 0000:04:00.0"
]


In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.wrappers.scikit_learn import KerasClassifier

In [3]:
train_data = pd.read_csv('../Kaggle_KKBox_Churn/training_feature_v2.csv')
test_data  = pd.read_csv('../Kaggle_KKBox_Churn/testing_feature_v2.csv')
from sklearn.preprocessing import LabelEncoder
catagory = ['payment_method_id', 'payment_plan_days', 'plan_list_price','actual_amount_paid', 'city','gender', 'registered_via']
encoder = {}
for c in catagory:
    print (c)
    le = LabelEncoder()
    le.fit(pd.concat([train_data[c],test_data[c]],axis=0))
    name = c+'_encoded'
    train_data[name]= le.transform(train_data.loc[:,c])
    test_data[name]= le.transform(train_data.loc[:,c])
    encoder[c] = le

# X = train_data.drop(columns=catagory) #pd > 0.20
# Y = test_data.drop(columns=catagory)
X = train_data.drop(catagory,axis=1)
Y = test_data.drop(catagory,axis=1)

X.drop('msno',axis=1,inplace=True)
Y.drop('msno',axis=1,inplace=True)

X = X.iloc[:,1:].values
Y = Y.iloc[:,0].values
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.2,random_state=0)

payment_method_id
payment_plan_days
plan_list_price
actual_amount_paid
city
gender
registered_via


In [9]:
### Baseline Model
import time
from sklearn.metrics import accuracy_score, log_loss
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

DNNbs = Sequential()
DNNbs.add(Dense(90, input_dim=33, kernel_initializer='normal', activation='linear'))
DNNbs.add(Dropout(0.2))
DNNbs.add(Dense(60, kernel_initializer='normal', activation='relu'))
DNNbs.add(Dropout(0.2))
DNNbs.add(Dense(30, kernel_initializer='normal', activation='relu'))
DNNbs.add(Dropout(0.2))
DNNbs.add(Dense(10, kernel_initializer='normal', activation='relu'))
DNNbs.add(Dropout(0.2))
DNNbs.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
DNNbs.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

start = time.time()
DNNbs.fit(sc.fit_transform(X_train), y_train, epochs=200, batch_size=64,verbose=0)
elapsed = time.time() - start

y_pred = DNNbs.predict(sc.transform(X_test))
y_pred_proba = DNNbs.predict_proba(sc.fit_transform(X_test))
logloss = log_loss(y_test,y_pred_proba)
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(y_test, predictions)

print("Accuracy: {:.2f}%".format(accuracy * 100.0))
print("LogLoss:  {:.4f}".format(logloss))
print("Total Time {:.2f}".format(elapsed))

192320/193687 [============================>.] - ETA: 0ss  2208/193687 [..............................] - ETA: 4s 

/share/apps/scikit-learn/0.18.1/intel/lib/python2.7/site-packages/sklearn/metrics/classification.py:1662: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
/share/apps/scikit-learn/0.18.1/intel/lib/python2.7/site-packages/sklearn/metrics/classification.py:1662: RuntimeWarning: invalid value encountered in multiply
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)


In [32]:
DNNbs.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 90)                3060      
_________________________________________________________________
dropout_1 (Dropout)          (None, 90)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 60)                5460      
_________________________________________________________________
dropout_2 (Dropout)          (None, 60)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 30)                1830      
_________________________________________________________________
dropout_3 (Dropout)          (None, 30)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 10)                310       
__________

In [47]:
sc.fit(X_train)
y_pred_proba = DNNbs.predict_proba(sc.transform(X_test))
logloss = log_loss(y_test,y_pred_proba)
predictions = [round(value) for value in y_pred_proba]
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: {:.2f}%".format(accuracy * 100.0))
print("LogLoss:  {:.4f}".format(logloss))

192352/193687 [============================>.] - ETA: 0ss  2240/193687 [..............................] - ETA: 4s 

In [67]:
def log_loss_scratch(y_test,y_pred):
    from numpy import ma
    y_pred = y_pred.squeeze()
    return (-(y_test*ma.log(y_pred)+(1-y_test)*ma.log(1-y_pred))).mean()
log_loss_scratch(y_test,y_pred)

0.17316379775430493